In [48]:
import requests
from bs4 import BeautifulSoup
import json
import re 
import urllib.parse

In [3]:
def getBranch(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Lỗi tải trang: {response.status_code}")
        return
    soup = BeautifulSoup(response.text, 'html.parser')
    result = []
    try:
        branches = soup.select('.yq')
        for branch in branches:
            nameBranch = branch.select_one('p > a[href]').text
            link = branch.select_one('a[href]').get('href')
            result.append({
                "name": nameBranch,
                "link": link
            })
    except Exception as e:
        print(f"Error: {e}")
    with open('branch.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)
        print("Dữ liệu đã được lưu vào file `branch.json`.")

In [9]:
getBranch('https://iprice.vn/coupons/stores/')

Dữ liệu đã được lưu vào file `branch.json`.


In [12]:
def getCategory(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Lỗi tải trang: {response.status_code}")
        return
    soup = BeautifulSoup(response.text, 'html.parser')
    result = []
    try:
        categories = soup.select('.uk.oY.pv3-4x.ph6-4x.bM.rg.bF.p3.rl.w-120.h-min-136')
        for category in categories:
            name = category.select_one('h3.bG.ac.ad.eg.tc.ic.aV.line-clamp.rp').text
            link = category.get('href')
            result.append({
                "name": name,
                "link": link
            })
    except Exception as e:
        print(f"Error: {e}")
    with open('category_voucher.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)
        print("Dữ liệu đã được lưu vào file `branch.json`.")

In [ ]:
getCategory('https://iprice.vn/coupons/')

In [55]:
def getVoucher(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Lỗi tải trang: {response.status_code}")
        return
    soup = BeautifulSoup(response.text, 'html.parser')
    result = []
    try:
        vouchers = soup.select('.l4.q8.gN.l7.bM.q8.zd.c-green.kq.nocode, .l4.q8.gN.l7.bM.q8.zd.c-orange.kq.code, .l4.q8.gN.l7.bM.q8.zd.c-blue.kq.nocode')
        for voucher in vouchers:
            name = voucher.select_one('.offer-text-border').text
            onClick = voucher.select_one('.dF.ek.lZ.ah.aU[onclick]')
            if onClick:
                onClick = onClick.get('onclick', '')
                # Sử dụng regex để trích xuất URL từ onclick
                match = re.search(r"openPopup\('[^']+',\s*'([^']+)'", onClick)
                if match:
                    link = match.group(1)
                    # Thay thế || thành : và | thành /
                    link = link.replace("||", "//").replace("|", "/")
                else:
                    link = 'N/A'
            else:
                link = 'N/A'
            info = voucher.select_one('p.a.en.rp.f1').text
            details = voucher.select_one('.dn.hidden-details.fq.cC.gL.gy.pl5-l-4x.yh.uP.dF')
            if details:
                # Lấy giá trị từ các phần tử con
                hieu_luc = details.select('.hk .bh.fM.bc.gray-8b.ac.en.hh')
                hieu_luc = hieu_luc[0].find_next('div').text.strip() if len(hieu_luc) > 0 else 'N/A'

                gioi_han = details.select('.hk .bh.fM.bc.gray-8b.ac.en.hh')
                gioi_han = gioi_han[1].find_next('div').text.strip() if len(gioi_han) > 1 else 'N/A'

                chi_tieu = details.select('.hk .bh.fM.bc.gray-8b.ac.en.hh')
                chi_tieu = chi_tieu[2].find_next('div').text.strip() if len(chi_tieu) > 2 else 'N/A'

                doi_tuong = details.select('.hk .bh.fM.bc.gray-8b.ac.en.hh')
                doi_tuong = doi_tuong[3].find_next('div').text.strip() if len(doi_tuong) > 3 else 'N/A'
            else:
                hieu_luc = 'N/A'
                gioi_han = 'N/A'
                chi_tieu = 'N/A'
                doi_tuong = 'N/A'
            result.append({
                "name": name,
                "link": link,
                "info": info,
                "hieu_luc": hieu_luc,
                "gioi_han": gioi_han,
                "chi_tieu": chi_tieu,
                "doi_tuong": doi_tuong,

            })
    except Exception as e:
        print(f"Error: {e}")
    with open('branch_voucher.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)
        print("Dữ liệu đã được lưu vào file")

In [ ]:
getVoucher("https://iprice.vn/coupons/top-20-ma-giam-gia/")

In [62]:
def getVoucherFromCate(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Lỗi tải trang: {response.status_code}")
        return
    soup = BeautifulSoup(response.text, 'html.parser')
    result = []
    try:
        vouchers = soup.select('.l4.q8.gN.l7.bM.q8.zd.c-green.kq.nocode, .l4.q8.gN.l7.bM.q8.zd.c-orange.kq.code, .l4.q8.gN.l7.bM.q8.zd.c-blue.kq.nocode')
        for voucher in vouchers:
            link_voucher = voucher.select_one('a.z-3.pa3-4x.bF.rl.tA.coupon-img-container.h-110-l[href]').get('href')
            onClick = voucher.select_one('.dF.ek.lZ.ah.aU[onclick]')
            if onClick:
                onClick = onClick.get('onclick', '')
                # Sử dụng regex để trích xuất URL từ onclick
                match = re.search(r"openPopup\('[^']+',\s*'([^']+)'", onClick)
                if match:
                    link = match.group(1)
                    # Thay thế || thành : và | thành /
                    link = link.replace("||", "//").replace("|", "/")
                else:
                    link = 'N/A'
            else:
                link = 'N/A'
            info = voucher.select_one('p.a.en.rp.f1').text
            details = voucher.select_one('.dn.hidden-details.fq.cC.gL.gy.pl5-l-4x.yh.uP.dF')
            if details:
                # Lấy giá trị từ các phần tử con
                hieu_luc = details.select('.hk .bh.fM.bc.gray-8b.ac.en.hh')
                hieu_luc = hieu_luc[0].find_next('div').text.strip() if len(hieu_luc) > 0 else 'N/A'

                gioi_han = details.select('.hk .bh.fM.bc.gray-8b.ac.en.hh')
                gioi_han = gioi_han[1].find_next('div').text.strip() if len(gioi_han) > 1 else 'N/A'

                chi_tieu = details.select('.hk .bh.fM.bc.gray-8b.ac.en.hh')
                chi_tieu = chi_tieu[2].find_next('div').text.strip() if len(chi_tieu) > 2 else 'N/A'

                doi_tuong = details.select('.hk .bh.fM.bc.gray-8b.ac.en.hh')
                doi_tuong = doi_tuong[3].find_next('div').text.strip() if len(doi_tuong) > 3 else 'N/A'
            else:
                hieu_luc = 'N/A'
                gioi_han = 'N/A'
                chi_tieu = 'N/A'
                doi_tuong = 'N/A'
            result.append({
                "link_voucher": link_voucher,
                "link": link,
                "info": info,
                "hieu_luc": hieu_luc,
                "gioi_han": gioi_han,
                "chi_tieu": chi_tieu,
                "doi_tuong": doi_tuong,

            })
    except Exception as e:
        print(f"Error: {e}")
    with open('branch_voucher.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)
        print("Dữ liệu đã được lưu vào file")

In [63]:
getVoucherFromCate('https://iprice.vn/coupons/')

Dữ liệu đã được lưu vào file
